In [86]:
import pandas as pd
import numpy as np

In [126]:
# load impressions for 2022
# df_02 = pd.read_csv('../datasets/impressions_02.csv')
# df_03 = pd.read_csv('../datasets/impressions_02.csv')
# df_q2 = pd.read_csv('../datasets/impressions_04_05_06.csv')
# df_q3 = pd.read_csv('../datasets/impressions_07_08_09.csv')
# df_q4 = pd.read_csv('../datasets/impressions_10_11_12.csv')
# dfs = [df_02, df_03, df_q2, df_q3, df_q4]
# impressions_df = pd.concat(dfs)

impressions_df = pd.read_csv('../datasets/impressions_12.csv')
impressions_df

,id,hostname,placement_ids,ad_width_chosen,ad_height_chosen,is_fixated,gaze_valid,total_fixation_duration
0,69de45c1-d112-477c-bb62-8fc577d2f5c2,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",180,150,False,False,0
1,6c40223e-fc34-4bf4-b294-bd09a87d142e,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",180,150,False,False,0
2,9fd6cc2c-ed1e-4a43-a4bb-b39971174499,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",180,150,False,False,0
3,c45c2aaa-b700-4c53-bdf5-0498fa5f1e2c,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",1180,240,False,False,0
4,5e96fb0e-2b25-4468-9257-72e3658efec3,apps.facebook.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",728,90,False,False,0
...,...,...,...,...,...,...,...,...
2127462,aa00970b-8d18-4ab8-b244-bd60f2bde1ee,hitta.se,"{""tag_id"":[""hitta_desktop_privatedetail_panora...",980,240,False,False,0
2127463,009906a1-bca5-430f-bd4d-12463974286b,hitta.se,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",980,240,False,False,0
2127464,0426f3cc-0643-47c9-99ca-2f5b8fe3d7d9,hitta.se,"{""tag_id"":[""hitta_desktop_privatedetail_panora...",980,240,False,False,0
2127465,3bfd2ada-7aa5-4296-a934-0688f13484bc,hitta.se,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",980,240,False,False,0


In [127]:
# adding size column
def get_impression_size(row):
    return f"{row['ad_width_chosen']}x{row['ad_height_chosen']}"

impressions_df['size'] = impressions_df.apply(lambda x: get_impression_size(x), axis=1)
impressions_df

,id,hostname,placement_ids,ad_width_chosen,ad_height_chosen,is_fixated,gaze_valid,total_fixation_duration,size
0,69de45c1-d112-477c-bb62-8fc577d2f5c2,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",180,150,False,False,0,180x150
1,6c40223e-fc34-4bf4-b294-bd09a87d142e,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",180,150,False,False,0,180x150
2,9fd6cc2c-ed1e-4a43-a4bb-b39971174499,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",180,150,False,False,0,180x150
3,c45c2aaa-b700-4c53-bdf5-0498fa5f1e2c,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",1180,240,False,False,0,1180x240
4,5e96fb0e-2b25-4468-9257-72e3658efec3,apps.facebook.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",728,90,False,False,0,728x90
...,...,...,...,...,...,...,...,...,...
2127462,aa00970b-8d18-4ab8-b244-bd60f2bde1ee,hitta.se,"{""tag_id"":[""hitta_desktop_privatedetail_panora...",980,240,False,False,0,980x240
2127463,009906a1-bca5-430f-bd4d-12463974286b,hitta.se,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",980,240,False,False,0,980x240
2127464,0426f3cc-0643-47c9-99ca-2f5b8fe3d7d9,hitta.se,"{""tag_id"":[""hitta_desktop_privatedetail_panora...",980,240,False,False,0,980x240
2127465,3bfd2ada-7aa5-4296-a934-0688f13484bc,hitta.se,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",980,240,False,False,0,980x240


In [128]:
impressions_df['placement_ids'].value_counts()

{"tag_id":[],"bam_ad_slot":["","",""]}                                           1684650
{"tag_id":["19499417"],"bam_ad_slot":["","",""]}                                   15443
{"tag_id":["19499443"],"bam_ad_slot":["","",""]}                                   13375
{"tag_id":["19499442"],"bam_ad_slot":["","",""]}                                   12323
{"tag_id":[],"bam_ad_slot":["insider","",""]}                                      11750
                                                                                  ...   
{"tag_id":[],"bam_ad_slot":["panorama","1","post-333967"]}                             1
{"tag_id":[],"bam_ad_slot":["panorama","3","post-333903"]}                             1
{"tag_id":[],"bam_ad_slot":["panorama","1","post-333923"]}                             1
{"tag_id":[],"bam_ad_slot":["mob","","post-333967"]}                                   1
{"tag_id":["20148742"],"bam_ad_slot":["articlemodule","6","articlemodule-2"]}          1
Name: placement_ids, 

In [129]:
import json

def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    # print(bam_ad_slots)
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        # print('.')
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret
    

def len_pids(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    return len(placement_id["tag_id"]) + len(final_bam_ad_slots)

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_len'] = impressions_df['placement_ids'].apply(len_pids)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)

In [130]:
# filter only impressions with int placement IDs
impressions_df = impressions_df[impressions_df['pid_type'] == 'int']
# convert placement ids that can be converted to the int to be int, not string
# type(impressions_df.iloc[0]['pid'])     # check this, right now they're strings
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))
impressions_df.head()

/home/markopejic/clear/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,hostname,placement_ids,ad_width_chosen,ad_height_chosen,is_fixated,gaze_valid,total_fixation_duration,size,pid,pid_len,pid_type
165,33a2ff2e-1319-46d1-9024-5e607ba7928f,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",1180,240,True,True,844,1180x240,19499440,1,int
166,ac0e3d54-9ac8-4b9e-adb7-b3acc4c3447e,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",1180,240,True,True,844,1180x240,19499440,1,int
167,948f36a9-e043-4f88-8b08-384b196aaf3e,aftonbladet.se,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",980,240,True,True,160,980x240,19499441,1,int
168,9c4daa63-3b5c-446a-904c-dfdff0b0323d,aftonbladet.se,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",250,480,False,True,0,250x480,19499420,1,int
169,9cb251c4-af3e-44e4-9c8b-f7ee62cbb238,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",180,150,False,True,0,180x150,19499443,1,int


In [131]:
temp_data = impressions_df[impressions_df['pid'] == 12005770]
temp_data['ad_width_chosen'].value_counts()

180    2
300    1
Name: ad_width_chosen, dtype: int64

In [132]:
impressions_df['pid'].value_counts()

19499417    15443
19499443    13375
19499442    12323
19499441    10056
19499428     9834
            ...  
28226268        1
20439164        1
27390241        1
28526776        1
27430846        1
Name: pid, Length: 3269, dtype: int64

In [133]:
# run through predictions
import os
import json


def _get_pred_map():
    pred_map = dict()
    
    for i in range(1, 13):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../predictions/2022/{i}'):
            for h in os.listdir(f'../predictions/2022/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../predictions/2022/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [134]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)
impressions_df

/home/markopejic/clear/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,hostname,placement_ids,ad_width_chosen,ad_height_chosen,is_fixated,gaze_valid,total_fixation_duration,size,pid,pid_len,pid_type,final_fixation
165,33a2ff2e-1319-46d1-9024-5e607ba7928f,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",1180,240,True,True,844,1180x240,19499440,1,int,True
166,ac0e3d54-9ac8-4b9e-adb7-b3acc4c3447e,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",1180,240,True,True,844,1180x240,19499440,1,int,True
167,948f36a9-e043-4f88-8b08-384b196aaf3e,aftonbladet.se,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",980,240,True,True,160,980x240,19499441,1,int,True
168,9c4daa63-3b5c-446a-904c-dfdff0b0323d,aftonbladet.se,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",250,480,False,True,0,250x480,19499420,1,int,False
169,9cb251c4-af3e-44e4-9c8b-f7ee62cbb238,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",180,150,False,True,0,180x150,19499443,1,int,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127280,c453ed4f-6aa3-4082-be78-fd834801020d,aftonbladet.se,"{""tag_id"":[""19499436""],""bam_ad_slot"":["""","""",""""]}",468,240,False,False,0,468x240,19499436,1,int,True
2127285,c0bc773a-4b9a-4115-b9c7-306c5846793a,uk.mail.yahoo.com,"{""tag_id"":[""26810648""],""bam_ad_slot"":["""","""",""""]}",180,150,False,False,0,180x150,26810648,1,int,False
2127286,4a155f89-33b0-4374-b4aa-c3b5c92d90a3,uk.mail.yahoo.com,"{""tag_id"":[""26810648""],""bam_ad_slot"":["""","""",""""]}",300,250,False,False,0,300x250,26810648,1,int,False
2127335,783231cf-b5d1-4bf1-a9f7-74c230140239,aftonbladet.se,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",250,480,False,False,0,250x480,19499420,1,int,False


In [135]:
# load omd dataset
# df_client_omd = pd.read_excel('../datasets/report - seller-publisher.xlsx')
# df_client_omd

In [136]:
# filter only impressions with tele2 placement IDs
# omd_placements = list(df_client_omd['Placement Id'].unique())
# omd_impressions = impressions_df[impressions_df['pid'].isin(omd_placements)]
# omd_impressions

In [137]:
impressions_df['total_fixation_duration'].value_counts()

0        232207
33          208
30          195
22          160
31          125
          ...  
11446         1
2207          1
1105          1
9401          1
24647         1
Name: total_fixation_duration, Length: 3071, dtype: int64

In [138]:
# groupby and aggregate
grouped_df = impressions_df.groupby(['hostname', 'pid', 'size']).apply(
    lambda x: pd.Series(
        {
            'num_impressions' : x['id'].count(),
            'num_fixations': x.loc[x['final_fixation'], 'id'].count(),
            'average_fixation_duration': x['total_fixation_duration'].mean(),
#             'width': x['ad_width_chosen'].mean(),
#             'height': x['ad_height_chosen'].mean()
        }
    )
)
grouped_df
# grouped_df[grouped_df.loc['average_fixation_duration'] != 0 & grouped_df.loc['num_impressions'] == 0]

num_impressions  \
hostname                      pid      size                        
1001fonts.com                 12005727 1180x240              1.0   
                              12005770 180x150               2.0   
                                       300x250               1.0   
                              12005772 180x150               2.0   
                                       250x480               3.0   
...                                                          ...   
zoopla.co.uk                  23831121 300x250               1.0   
zt2downloadlibrary.fandom.com 21645015 300x250               1.0   
                              26340171 160x600               1.0   
                                       180x150               1.0   
                              27472329 180x150               1.0   

                                                 num_fixations  \
hostname                      pid      size                      
1001fonts.com                 12005727 1180x240            0.0   
                              12005770 180x150             0.0   
                                       300x250             0.0   
                              12005772 180x150             0.0   
                                       250x480             0.0   
...                                                        ...   
zoopla.co.uk                  23831121 300x250             0.0   
zt2downloadlibrary.fandom.com 21645015 300x250             0.0   
                              26340171 160x600             0.0   
                                       180x150             0.0   
                              27472329 180x150             0.0   

                                                 average_fixation_duration  
hostname                      pid      size                                 
1001fonts.com                 12005727 1180x240                        0.0  
                              12005770 180x150                         0.0  
                                       300x250                         0.0  
                              12005772 180x150                         0.0  
                                       250x480                         0.0  
...                                                                    ...  
zoopla.co.uk                  23831121 300x250                         0.0  
zt2downloadlibrary.fandom.com 21645015 300x250                         0.0  
                              26340171 160x600                         0.0  
                                       180x150                         0.0  
                              27472329 180x150                         0.0  

[6715 rows x 3 columns]

In [139]:
grouped_df['ratio'] = round(grouped_df['num_fixations'] / grouped_df['num_impressions'], 4)
grouped_df['ratio_percent'] = round(grouped_df['ratio'] * 100, 2)
grouped_df['average_fixation_duration'] = round(grouped_df['average_fixation_duration'], 2)
grouped_df

num_impressions  \
hostname                      pid      size                        
1001fonts.com                 12005727 1180x240              1.0   
                              12005770 180x150               2.0   
                                       300x250               1.0   
                              12005772 180x150               2.0   
                                       250x480               3.0   
...                                                          ...   
zoopla.co.uk                  23831121 300x250               1.0   
zt2downloadlibrary.fandom.com 21645015 300x250               1.0   
                              26340171 160x600               1.0   
                                       180x150               1.0   
                              27472329 180x150               1.0   

                                                 num_fixations  \
hostname                      pid      size                      
1001fonts.com                 12005727 1180x240            0.0   
                              12005770 180x150             0.0   
                                       300x250             0.0   
                              12005772 180x150             0.0   
                                       250x480             0.0   
...                                                        ...   
zoopla.co.uk                  23831121 300x250             0.0   
zt2downloadlibrary.fandom.com 21645015 300x250             0.0   
                              26340171 160x600             0.0   
                                       180x150             0.0   
                              27472329 180x150             0.0   

                                                 average_fixation_duration  \
hostname                      pid      size                                  
1001fonts.com                 12005727 1180x240                        0.0   
                              12005770 180x150                         0.0   
                                       300x250                         0.0   
                              12005772 180x150                         0.0   
                                       250x480                         0.0   
...                                                                    ...   
zoopla.co.uk                  23831121 300x250                         0.0   
zt2downloadlibrary.fandom.com 21645015 300x250                         0.0   
                              26340171 160x600                         0.0   
                                       180x150                         0.0   
                              27472329 180x150                         0.0   

                                                 ratio  ratio_percent  
hostname                      pid      size                            
1001fonts.com                 12005727 1180x240    0.0            0.0  
                              12005770 180x150     0.0            0.0  
                                       300x250     0.0            0.0  
                              12005772 180x150     0.0            0.0  
                                       250x480     0.0            0.0  
...                                                ...            ...  
zoopla.co.uk                  23831121 300x250     0.0            0.0  
zt2downloadlibrary.fandom.com 21645015 300x250     0.0            0.0  
                              26340171 160x600     0.0            0.0  
                                       180x150     0.0            0.0  
                              27472329 180x150     0.0            0.0  

[6715 rows x 5 columns]

In [140]:
result_df = grouped_df.reset_index()
result_df.rename(columns={'pid': 'placement_id'}, inplace=True)
result_df

,hostname,placement_id,size,num_impressions,num_fixations,average_fixation_duration,ratio,ratio_percent
0,1001fonts.com,12005727,1180x240,1.0,0.0,0.0,0.0,0.0
1,1001fonts.com,12005770,180x150,2.0,0.0,0.0,0.0,0.0
2,1001fonts.com,12005770,300x250,1.0,0.0,0.0,0.0,0.0
3,1001fonts.com,12005772,180x150,2.0,0.0,0.0,0.0,0.0
4,1001fonts.com,12005772,250x480,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
6710,zoopla.co.uk,23831121,300x250,1.0,0.0,0.0,0.0,0.0
6711,zt2downloadlibrary.fandom.com,21645015,300x250,1.0,0.0,0.0,0.0,0.0
6712,zt2downloadlibrary.fandom.com,26340171,160x600,1.0,0.0,0.0,0.0,0.0
6713,zt2downloadlibrary.fandom.com,26340171,180x150,1.0,0.0,0.0,0.0,0.0


In [141]:
# load omd dataset
df_client_omd = pd.read_excel('../datasets/report - seller-publisher.xlsx')
df_client_omd

,Seller,Seller Name,Publisher Id,Publisher Name,Placement Id,Placement Name,Member Currency,Imps
0,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499475,SE-Aftonbladet-wph-Front-Panorama_1,SEK,249128
1,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499490,SE-Aftonbladet-wph-Front-Toppanorama,SEK,184180
2,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499428,SE-Aftonbladet-wde-Front-Modul_1,SEK,217728
3,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499363,SE-Aftonbladet-apa-Front-Toppanorama,SEK,92770
4,9943,Schibsted Sweden,1403585,SE-Aftonbladet,19499470,SE-Aftonbladet-wph-Article-Toppanorama,SEK,131159
...,...,...,...,...,...,...,...,...
281,9943,Schibsted Sweden,1431700,SE-Bytbil,14746565,SE-BytBil-wph-Sok_Lista_Fordon_Panorama_2,SEK,1481
282,12099,BONNIER NEWS AB,1856396,MM Bandypuls,20148910,Mob,SEK,1122
283,9943,Schibsted Sweden,1403613,SE-TV,19499789,SE-TV-wde-Other-Toppanorama,SEK,643
284,12099,BONNIER NEWS AB,1856377,BNS Expressen,20615026,Mob_top,SEK,878


In [142]:
# merge impressions with clients data
result_df = result_df.merge(df_client_omd,  left_on='placement_id', right_on='Placement Id', how='left')
result_df

,hostname,placement_id,size,num_impressions,num_fixations,average_fixation_duration,ratio,ratio_percent,Seller,Seller Name,Publisher Id,Publisher Name,Placement Id,Placement Name,Member Currency,Imps
0,1001fonts.com,12005727,1180x240,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001fonts.com,12005770,180x150,2.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001fonts.com,12005770,300x250,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001fonts.com,12005772,180x150,2.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001fonts.com,12005772,250x480,3.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6710,zoopla.co.uk,23831121,300x250,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6711,zt2downloadlibrary.fandom.com,21645015,300x250,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6712,zt2downloadlibrary.fandom.com,26340171,160x600,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6713,zt2downloadlibrary.fandom.com,26340171,180x150,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
result_df['Placement Name'].value_counts()

Outsider                                15
Panorama                                14
Insider                                 13
Module                                  13
SE-Aftonbladet-wde-Article-Insider_1     9
                                        ..
SE-SvD-wde-Front-Widescreen_2            2
SE-Klart-wph-Other-Panorama_2            1
SE-Klart-wph-Other-Panorama_1            1
SE-Klart-wph-Front-Toppanorama           1
SE-SvD-wph-Article-Panorama_3            1
Name: Placement Name, Length: 119, dtype: int64

In [144]:
result_df = result_df[[
    'Seller Name', 
    'Publisher Name', 
    'hostname', 
    'placement_id',
    'size',
#     'num_impressions', 
#     'num_fixations', 
    'ratio', 
#    'ratio_percent', 
    'average_fixation_duration']]
result_df.rename(columns={'Seller Name': 'seller_name', 'Publisher Name': 'publisher_name'}, inplace=True)
result_df

/home/markopejic/clear/lib/python3.7/site-packages/pandas/core/frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration
0,NaN,NaN,1001fonts.com,12005727,1180x240,0.0,0.0
1,NaN,NaN,1001fonts.com,12005770,180x150,0.0,0.0
2,NaN,NaN,1001fonts.com,12005770,300x250,0.0,0.0
3,NaN,NaN,1001fonts.com,12005772,180x150,0.0,0.0
4,NaN,NaN,1001fonts.com,12005772,250x480,0.0,0.0
...,...,...,...,...,...,...,...
6710,NaN,NaN,zoopla.co.uk,23831121,300x250,0.0,0.0
6711,NaN,NaN,zt2downloadlibrary.fandom.com,21645015,300x250,0.0,0.0
6712,NaN,NaN,zt2downloadlibrary.fandom.com,26340171,160x600,0.0,0.0
6713,NaN,NaN,zt2downloadlibrary.fandom.com,26340171,180x150,0.0,0.0


In [145]:
# sort on seller_name and publisher_name
result_df.sort_values(by=['publisher_name', 'seller_name'], inplace=True)
result_df.rename({'ratio_percent': 'ratio'}, inplace=True)
result_df

/home/markopejic/clear/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/markopejic/clear/lib/python3.7/site-packages/pandas/core/frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration
2134,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x240,1.0000,580.00
2130,BONNIER NEWS AB,BMB Amelia,expressen.se,20148868,980x360,0.7778,932.33
2132,BONNIER NEWS AB,BMB Mama,expressen.se,20148873,980x360,0.0000,0.00
3900,BONNIER NEWS AB,BMB Tailsweep,monasuniversum.se,20148888,180x150,0.0000,0.00
1430,BONNIER NEWS AB,BNS Dagens industri,di.se,20148739,180x150,0.0000,0.00
...,...,...,...,...,...,...,...
6710,NaN,NaN,zoopla.co.uk,23831121,300x250,0.0000,0.00
6711,NaN,NaN,zt2downloadlibrary.fandom.com,21645015,300x250,0.0000,0.00
6712,NaN,NaN,zt2downloadlibrary.fandom.com,26340171,160x600,0.0000,0.00
6713,NaN,NaN,zt2downloadlibrary.fandom.com,26340171,180x150,0.0000,0.00


In [146]:
# highlight rows with invalid data combinations
# invalid combinations:
#    * ratio == 0 and average_fixation_duration > 0
#    * ratio > 0 and average_fixation_duration == 0

def highlight_rows(row):
    if ((row['ratio'] == 0 and row['average_fixation_duration'] > 0) or (row['ratio'] > 0 and row['average_fixation_duration'] == 0)):
        return False
        # return['background-color: orange']*5
    else:
        return True
        # return['background-color: white']*5

# display(result_df.style.apply(highlight_rows, axis=1))
result_df['eyetracker_on'] = result_df.apply(lambda row: highlight_rows(row), axis=1)
result_df

/home/markopejic/clear/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration,eyetracker_on
2134,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x240,1.0000,580.00,True
2130,BONNIER NEWS AB,BMB Amelia,expressen.se,20148868,980x360,0.7778,932.33,True
2132,BONNIER NEWS AB,BMB Mama,expressen.se,20148873,980x360,0.0000,0.00,True
3900,BONNIER NEWS AB,BMB Tailsweep,monasuniversum.se,20148888,180x150,0.0000,0.00,True
1430,BONNIER NEWS AB,BNS Dagens industri,di.se,20148739,180x150,0.0000,0.00,True
...,...,...,...,...,...,...,...,...
6710,NaN,NaN,zoopla.co.uk,23831121,300x250,0.0000,0.00,True
6711,NaN,NaN,zt2downloadlibrary.fandom.com,21645015,300x250,0.0000,0.00,True
6712,NaN,NaN,zt2downloadlibrary.fandom.com,26340171,160x600,0.0000,0.00,True
6713,NaN,NaN,zt2downloadlibrary.fandom.com,26340171,180x150,0.0000,0.00,True


In [147]:
result_df['eyetracker_on'].value_counts()
result_df['ratio'] *= 100

/home/markopejic/clear/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [148]:
result_df.to_csv('omg_performance_report_2022.csv', index=False)

In [149]:
len(result_df['ratio'].value_counts())

273

In [150]:
result_df

,seller_name,publisher_name,hostname,placement_id,size,ratio,average_fixation_duration,eyetracker_on
2134,BONNIER NEWS AB,BMB Allt om mat,expressen.se,20148892,980x240,100.00,580.00,True
2130,BONNIER NEWS AB,BMB Amelia,expressen.se,20148868,980x360,77.78,932.33,True
2132,BONNIER NEWS AB,BMB Mama,expressen.se,20148873,980x360,0.00,0.00,True
3900,BONNIER NEWS AB,BMB Tailsweep,monasuniversum.se,20148888,180x150,0.00,0.00,True
1430,BONNIER NEWS AB,BNS Dagens industri,di.se,20148739,180x150,0.00,0.00,True
...,...,...,...,...,...,...,...,...
6710,NaN,NaN,zoopla.co.uk,23831121,300x250,0.00,0.00,True
6711,NaN,NaN,zt2downloadlibrary.fandom.com,21645015,300x250,0.00,0.00,True
6712,NaN,NaN,zt2downloadlibrary.fandom.com,26340171,160x600,0.00,0.00,True
6713,NaN,NaN,zt2downloadlibrary.fandom.com,26340171,180x150,0.00,0.00,True
